In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import scipy.stats as sts
from scipy.optimize import root

 ## BS Option Pricing Function

In [2]:
class BS_euro_option :
    def call(S, K, T, r, sigma, div = 0):

        d1 = (np.log(S / K) + (r -div + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        call = (S * sts.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * sts.norm.cdf(d2, 0.0, 1.0))

        return call
    
    def put(S, K, T, r, sigma, div = 0):

        d1 = (np.log(S / K) + (r -div + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma*np.sqrt(T)
        put = (K * np.exp(-r * T) * sts.norm.cdf(-d2, 0.0, 1.0) - S * sts.norm.cdf(-d1, 0.0, 1.0))

        return put

In [12]:
BS_euro_option.call(101.17, 100, 1, 0.01, 0.45)

18.92039322427228

 ## BS implied Vol

In [3]:
class BS_implied_vol :
    def call(S,K,T,r,div,Price) :
        def fun(x,S,K,T,r,div,P) :
            return BS_euro_option.call(S,K,T,r,x,div) - P
        x0 = 0.5
        x = root(fun, x0, args = (S, K, T, r, div, Price))
        return x.x
    def put(S,K,T,r,div,Price) :
        def fun(x,S,K,T,r,div,P) :
            return BS_euro_option.put(S,K,T,r,x,div) - P
        x0 = 0.5
        x = root(fun, x0, args = (S, K, T, r, div, Price))
        return x.x

In [17]:
BS_implied_vol.call(S = 101.17,K = 100,T = 1,r = 0.01,div = 0, Price = 18.9203932)

array([0.45])

 # Binomial Option Pricing Function

## Cox Ross rubinstein
\begin{align}
\ u = e^{\sigma \sqrt{\Delta t} },\
\ d = \frac{1}{u} , \
\ p = \frac{e^{(r-\delta)\Delta t}\ -d}{u-d} \
\\
\end{align}
## Normal Step
\begin{align}
\ u = e^{(r-\delta)\Delta t + \sigma \sqrt{\Delta t} },\
\ d = e^{(r-\delta)\Delta t - \sigma \sqrt{\Delta t} }, \
\ p = \frac{e^{(r-\delta)\Delta t}\ -d}{u-d} \
\end{align}
## Stochastic Step
\begin{align}
\ u = e^{(r-\delta-\frac{1}{2} \sigma^2 )\Delta t + \sigma \sqrt{\Delta t} },\
\ d = e^{(r-\delta-\frac{1}{2} \sigma^2 )\Delta t - \sigma \sqrt{\Delta t} } , \
\ p = \frac{e^{(r-\delta)\Delta t}\ -d}{u-d} \
\end{align}
## Removing Nonlinearity (Leisen and Reimer 1996)
\begin{align}
\ p = h(d_2 ) , \ p^{\star} = h(d_1) \\
\\
\ d_1 = \frac{ln(S/K) + (r-\delta + \frac{1}{2} \sigma^2 ) T \ }{\sigma \sqrt{T}}, \ d_2 = d_1 - \sigma \sqrt{T} \\ \\
\ h(x) = \frac{1}{2} + sign(x) \sqrt{\frac{1}{4} - \frac{1}{4}exp[-(\frac{x}{N+\frac{1}{3}})^2(N+\frac{1}{6})]}\\
\ u = e^{(r-\delta)\Delta t  } \frac{p^\star}{p} ,\
\ d =  \frac{e^{(r-\delta)\Delta t  } - pu }{1-p} \
\end{align}


In [4]:
class Risk_neutral_u_d_p :
    
    ##########################################################################
    # You have to select The Model of Stock Movement or Dividend Processing ##
    ##########################################################################
    def continuous_dividends(sigma, T, N, Model = 'CRR',div = 0, r=0) :
        dt = T/N
        if Model.lower() == 'crr' :
            u = np.exp( sigma * np.sqrt(dt))
            d = 1/u

        elif Model.lower() == 'normal' :
            u = np.exp( (r-div)*dt + sigma * np.sqrt(dt))
            d = np.exp( (r-div)*dt - sigma * np.sqrt(dt))

        else :
            u = np.exp( (r-div-0.5*sigma**2)*dt + sigma * np.sqrt(dt))
            d = np.exp( (r-div-0.5*sigma**2)*dt - sigma * np.sqrt(dt))
            
        p = (np.exp((r-div)*dt)-d)/(u-d)
        
        return u,d,p
    
    def Removing_nomlinearity(sigma,T,N,r , div = 0) :
        dt = T/N
        d1 = (np.log(S/K) + (r-div+0.5*sigma**2)*T) / (sigma * np.sqrt(T))
        d2 = (np.log(S/K) + (r-div-0.5*sigma**2)*T) / (sigma * np.sqrt(T))
        q_star = 0.5 + np.sign(d1) * np.sqrt( 0.25 - 0.25 * np.exp( - (d1/(N+1/3))**2 * (N+1/6)  )  ) 
        q = 0.5 + np.sign(d2) * np.sqrt( 0.25 - 0.25 * np.exp( - (d2/(N+1/3))**2 * (N+1/6)  )  ) 
        
        u = np.exp( (r-div)*dt  ) * q_star/q
        d = (np.exp(  (r-div)*dt ) - q*u)/(1-q)
        p = (np.exp((r-div)*dt)-d)/(u-d)
        return u, d, p
    
def make_S_tree_u_d_p(S,K,T,N,sigma,r , Model = 'CRR',div = 0 , Removing_nonlinearity = False) :

    ####################################################
    ## This Function is used to make the Stock's Tree ##
    ####################################################
    
    dt = T/N
    if  Removing_nonlinearity == False :
        u,d,p = Risk_neutral_u_d_p.continuous_dividends(sigma = sigma , T = T , N = N , Model = Model , r = r, div = div)
    elif Removing_nonlinearity == True :
        u,d,p = Risk_neutral_u_d_p.Removing_nomlinearity(sigma = sigma , T = T , N = N  , r = r, div = div)

    Temp = np.ones((N+1,N+1))
    Temp[:,0] = 0
    Temp1 = Temp.cumsum(1)
    Te = Temp.copy()
    Te[0,:] = 0
    
    num_u = np.triu(Temp1 - Te.cumsum(0))
    num_d = np.triu(Temp1 - num_u)
    S_Tree = S * np.triu((u**num_u) * (d**num_d))
    return u,d,p ,S_Tree

In [5]:
class binomial_option :
    
    def Call(S,K,T,N,sigma,r , Model = 'CRR' , div = 0 , Removing_nonlinearity = False) :        
        u,d,p,S_Tree = make_S_tree_u_d_p(S,K,T,N,sigma,r , Model = Model, div= div , Removing_nonlinearity= Removing_nonlinearity)
        
        A = np.ones((N+1,1))
        A[0] = 0
        A = A.cumsum(0).astype(np.int64)
        B = np.rot90(np.rot90(A))

        C_exercise_Value = np.maximum( S_Tree - K , np.zeros(np.shape(S_Tree)))
        C = (C_exercise_Value[:,-1:] * sts.binom.pmf(B, N, p).round(7)).sum()  *  np.exp(-r*T)
        return C

    def Put(S,K,T,N,sigma,r , Model = 'CRR', div = 0, Removing_nonlinearity = False) :        
        u,d,p,S_Tree = make_S_tree_u_d_p(S,K,T,N,sigma,r , Model = Model, div = div , Removing_nonlinearity=Removing_nonlinearity)

        A = np.ones((N+1,1))
        A[0] = 0
        A = A.cumsum(0).astype(np.int64)
        B = np.rot90(np.rot90(A))

        P_exercise_Value = np.maximum( K- S_Tree , np.zeros(np.shape(S_Tree)))
        P = (P_exercise_Value[:,-1:] * sts.binom.pmf(B, N, p).round(7)).sum()  *  np.exp(-r*T)
        return P
    
    def Tree_C(S,K,T,N,sigma,r,Model = 'CRR', div = 0, For_American = False , Removing_nonlinearity = False) :
        u,d,p,S_Tree = make_S_tree_u_d_p(S,K,T,N,sigma,r , Model = Model,div = div , Removing_nonlinearity=Removing_nonlinearity)
        A = np.ones((N+1,1))
        A[0] = 0
        A = A.cumsum(0).astype(np.int64)
        B = np.rot90(np.rot90(A))        
        C_exercise_Value = np.maximum( S_Tree - K , np.zeros(np.shape(S_Tree)))
        Temp_array= np.zeros(S_Tree.shape)
        dt = T/N
        
        if For_American == False :
            Temp_array[:,0] = C_exercise_Value[:,-1]
            for i in range(1,len(Temp_array)) :
                Temp_array[:-i,i] = (Temp_array[ :-1, i-1] * p  + (1-p) * Temp_array[1:, i-1])[ :len(Temp_array) - i] * np.exp(-r * dt)        
        else :
            Temp_array[:,0] = C_exercise_Value[:,-1]
            Temp_array[:-1,1] = BS_euro_option.call(S = S_Tree[:,-2][:-1] , K = K , T = dt, sigma = sigma, r = r)
            for i in range(2,len(Temp_array)) :
                Temp_array[:-i,i] = (Temp_array[ :-1, i-1] * p  + (1-p) * Temp_array[1:, i-1])[ :len(Temp_array) - i] * np.exp(-r * dt)        
        Option_Value_Tree = Temp_array[:,::-1]
        return Option_Value_Tree
    
    def Tree_P(S,K,T,N,sigma,r,Model = 'CRR', div = 0, For_American = False , Removing_nonlinearity = False) :
        u,d,p,S_Tree = make_S_tree_u_d_p(S,K,T,N,sigma,r , Model = Model,div = div , Removing_nonlinearity= Removing_nonlinearity)
        A = np.ones((N+1,1))
        A[0] = 0
        A = A.cumsum(0).astype(np.int64)
        B = np.rot90(np.rot90(A))        
        P_exercise_Value = np.maximum(  K - S_Tree , np.zeros(np.shape(S_Tree)))
        Temp_array= np.zeros(S_Tree.shape)
        dt = T/N
        
        if For_American == False :
            Temp_array[:,0] = P_exercise_Value[:,-1]
            for i in range(1,len(Temp_array)) :
                Temp_array[:-i,i] = (Temp_array[ :-1, i-1] * p  + (1-p) * Temp_array[1:, i-1])[ :len(Temp_array) - i] * np.exp(-r * dt)        
        else :
            Temp_array[:,0] = P_exercise_Value[:,-1]
            Temp_array[:-1,1] = BS_euro_option.put(S = S_Tree[:,-2][:-1] , K = K , T = dt, sigma = sigma, r = r)
            for i in range(2,len(Temp_array)) :
                Temp_array[:-i,i] = (Temp_array[ :-1, i-1] * p  + (1-p) * Temp_array[1:, i-1])[ :len(Temp_array) - i] * np.exp(-r * dt)        
            
        Option_Value_Tree = Temp_array[:,::-1]
        return Option_Value_Tree

    def American_C(S,K,T,N,sigma,r,Model = 'CRR', div = 0, Removing_nonlinearity = False) :
        Option_Value_Tree =binomial_option.Tree_C(S,K,T,N,sigma,r,Model, div, For_American = True , Removing_nonlinearity = Removing_nonlinearity) 
        return Option_Value_Tree[0,0]
    
    def American_P(S,K,T,N,sigma,r,Model = 'CRR', div = 0, Removing_nonlinearity = False) :
        Option_Value_Tree =binomial_option.Tree_P(S,K,T,N,sigma,r,Model, div, For_American = True , Removing_nonlinearity = Removing_nonlinearity) 
        return Option_Value_Tree[0,0]


In [6]:
class Binomial_implied_vol :
    def call(S,K,T,N,r ,Price, Model = 'CRR' , div = 0 , Removing_nonlinearity = False) :
        def fun(x,S,K,T,N,r ,P, Model , div , Removing_nonlinearity ) :
            return binomial_option.Call(S,K,T,N,x,r , Model , div , Removing_nonlinearity ) - P
        x0 = 0.5
        x = root(fun, x0, args = (S,K,T,N,r ,Price, Model , div , Removing_nonlinearity ))
        return x.x
    def put(S,K,T,N,r ,Price, Model = 'CRR' , div = 0 , Removing_nonlinearity = False) :
        def fun(x,S,K,T,N,r ,P, Model , div , Removing_nonlinearity ) :
            return binomial_option.Put(S,K,T,N,x,r , Model , div , Removing_nonlinearity ) - P
        x0 = 0.5
        x = root(fun, x0, args = (S,K,T,N,r ,Price, Model , div , Removing_nonlinearity ))
        return x.x
    def american_call(S,K,T,N,r ,Price, Model = 'CRR' , div = 0 , Removing_nonlinearity = False) :
        def fun(x,S,K,T,N,r ,P, Model , div , Removing_nonlinearity ) :
            return binomial_option.American_C(S,K,T,N,x,r , Model , div , Removing_nonlinearity ) - P
        x0 = 0.5
        x = root(fun, x0, args = (S,K,T,N,r ,Price, Model , div , Removing_nonlinearity ))
        return x.x
    def american_put(S,K,T,N,r ,Price, Model = 'CRR' , div = 0 , Removing_nonlinearity = False) :
        def fun(x,S,K,T,N,r ,P, Model , div , Removing_nonlinearity ) :
            return binomial_option.American_P(S,K,T,N,x,r , Model , div , Removing_nonlinearity ) - P
        x0 = 0.5
        x = root(fun, x0, args = (S,K,T,N,r ,Price, Model , div , Removing_nonlinearity ))
        return x.x        
        

 ## European Option Price

In [7]:
S = 101.17
Model = 'CRR'
div = 0
Removing_nonlinearity = False
K = 100
r = 0.01
sigma = 0.45
T = 1
N = 100
Price = binomial_option.Call(S,K,T,N,sigma,r)
Price

18.91521103498599

 ## Implied Vol of European Option

In [8]:
Binomial_implied_vol.call(S,K,T,N,r,Price)

array([0.45])

 ## American Option Price

In [9]:
Price2 = binomial_option.American_C(S,K,T,N,sigma,r)
Price2

18.93429906333072

 ## Implied Vol of American Option

In [10]:
Binomial_implied_vol.american_call(S,K,T,N,r,Price2)

array([0.45])